In [1]:
import numpy as np

import os
from tqdm import tqdm
import gc
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
f'Tensorflow version: {tf.__version__}'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

'Tensorflow version: 2.12.0'

In [2]:
batch_size = 16
image_size = 480
samples = 4096

In [6]:
datasets = ["/kaggle/input/indian-birds-250/training_data_6/training_data_6","/kaggle/input/indian-birds-250/training_data_7/training_data_7","/kaggle/input/indian-birds-250/training_data_8/training_data_8","/kaggle/input/indian-birds-250/training_data_9/training_data_9","/kaggle/input/indian-birds-250/training_data_10/training_data_10"]
# datasets=[]
# datasets.extend(tf.io.gfile.glob("/kaggle/input/indian-birds-250/training_data_[^0-5]/*"))
print(len(datasets))
print(datasets)

5
['/kaggle/input/indian-birds-250/training_data_6/training_data_6', '/kaggle/input/indian-birds-250/training_data_7/training_data_7', '/kaggle/input/indian-birds-250/training_data_8/training_data_8', '/kaggle/input/indian-birds-250/training_data_9/training_data_9', '/kaggle/input/indian-birds-250/training_data_10/training_data_10']


In [7]:
train_ds = None
val_ds = None
class_names = []
increment = 250

for i,ds in enumerate(datasets):
    tds = image_dataset_from_directory(
        ds,
        seed=123,
        image_size=(image_size, image_size),
        batch_size=batch_size
    )

    class_names.extend(tds.class_names)

    tds = tds.map(lambda x, y: (x, y + increment))
    if train_ds is None:
        train_ds = tds
    else:
        train_ds = train_ds.concatenate(tds)

    if ds != datasets[-1]:
        increment += len(os.listdir(datasets[i+1]))

Found 37080 files belonging to 50 classes.
Found 33823 files belonging to 50 classes.
Found 45354 files belonging to 50 classes.
Found 35575 files belonging to 50 classes.
Found 39245 files belonging to 50 classes.


In [8]:
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def image_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))

def serialize_example(feature0, feature1):
    feature = {
      'feature0': image_feature(feature0),
      'feature1': int64_feature(feature1)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()
    
def tf_serialize_example(f0,f1):
    tf_string = tf.py_function(serialize_example, (f0, f1), tf.string)
    return tf.reshape(tf_string, ())

In [9]:
if not os.path.exists('tfrecords'):
    os.mkdir('tfrecords')

In [10]:
print(len(class_names))
print(class_names)

250
['Aberrant Bush Warbler', 'Alpine Swift', 'Alpine Thrush', 'Altai Accentor', 'Amur Falcon', 'Andaman Coucal', 'Andaman Crake', 'Andaman Cuckooshrike', 'Andaman Masked-Owl', 'Andaman Nightjar', 'Andaman Scops-Owl', 'Andaman Serpent-Eagle', 'Ashy Wood-Pigeon', 'Ashy-throated Warbler', 'Asian Barred Owlet', 'Asian Brown Flycatcher', 'Asian Glossy Starling', 'Asian House-Martin', 'Assam Laughingthrush', 'Banded Bay Cuckoo', 'Bank Swallow', 'Bar-tailed Treecreeper', 'Bar-winged Wren-Babbler', 'Barn Swallow', 'Barred Buttonquail', 'Beautiful Sibia', 'Bengal Bushlark', 'Bengal Florican', 'Black Baza', 'Black Bulbul', 'Black Francolin', 'Black Redstart', 'Black Stork', 'Black-and-orange Flycatcher', 'Black-and-yellow Grosbeak', 'Black-backed Dwarf-Kingfisher', 'Black-bellied Plover', 'Black-breasted Parrotbill', 'Black-breasted Thrush', 'Black-capped Kingfisher', 'Black-chinned Babbler', 'Black-crested Bulbul', 'Black-eared Shrike-Babbler', 'Black-faced Laughingthrush', 'Black-faced Warble

In [11]:
with open('cnames.txt','w') as f:
    f.write(str(class_names))

In [12]:
train_ds = train_ds.unbatch()

In [13]:
del tds,increment,datasets,ds
gc.collect()

347

In [14]:
serialized=list()

for images, labels in tqdm(train_ds):
    serialized.append(serialize_example(images.numpy(),labels.numpy()))

191077it [32:16, 98.69it/s] 


In [15]:
del train_ds
gc.collect()

1422

In [16]:
l = len(serialized)

In [17]:
del f,images,labels
gc.collect()

0

In [18]:
ix = l//samples
serialized_temp = list()
for i in range(ix):
    serialized_temp.append(serialized[i*samples:(i+1)*samples])
serialized_temp.append(serialized[(ix)*samples:])

In [19]:
del serialized, ix

In [20]:
%%time
for i,s in enumerate(serialized_temp):
    filename = os.path.join("tfrecords","batch_2_{:02.0f}.tfrec".format(i))
    with tf.io.TFRecordWriter(filename) as writer:
        for rec in s:
            writer.write(rec)
del i,s,serialized_temp,writer
gc.collect()

CPU times: user 9.22 s, sys: 28.3 s, total: 37.5 s
Wall time: 1min 2s


0

In [ ]:
%%time
file_idx = 0
for _ in range(c):
    print(f"Batch: {file_idx}/{c}")
    filename = os.path.join('tfrecords','batch_{:02.0f}_{:02.0f}.tfrec'.format(idx,file_idx))
    ds = train_ds.skip(file_idx*1).take(1)
    for images, labels in ds:
        with tf.io.TFRecordWriter(filename) as writer:
            pbar = tqdm(range(batch_size))
            for i in pbar:
                pbar.set_description(f"Processing batch: {file_idx}")
                try:
                    serialized_example = serialize_example(images.numpy()[i], labels.numpy()[i])
                    writer.write(serialized_example)
                except:
                    pass
    file_idx += 1